In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
df = pd.read_csv("/home/palnika-chavan/Downloads/Dataset/cardekho_imputated.csv")

In [4]:
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [5]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [6]:
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [9]:
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)
        
    ],remainder='passthrough'
    
)
X=preprocessor.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((12328, 15), (3083, 15))

In [11]:
from sklearn.ensemble import AdaBoostRegressor

In [12]:
from sklearn import metrics
def evaluate_model(y_true, y_pred):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    r2 = metrics.r2_score(y_true, y_pred)
    return mae, rmse, r2

In [13]:
model = AdaBoostRegressor()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

In [14]:
print('Model performance for Training set')
print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
print("- R2 Score: {:.4f}".format(model_train_r2))

print('----------------------------------')

print('Model performance for Test set')
print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
print("- R2 Score: {:.4f}".format(model_test_r2))
print('----------------------------------')

Model performance for Training set
- Root Mean Squared Error: 488985.1374
- Mean Absolute Error: 396145.3838
- R2 Score: 0.7052
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 512356.6933
- Mean Absolute Error: 409059.4806
- R2 Score: 0.6513
----------------------------------


In [15]:
ada_params={
    "n_estimators":[50,60,70,80],
    "loss":['linear','square','exponential']
}
randomcv_models = [
                ("Adaboost",AdaBoostRegressor(),ada_params)
]

In [17]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END .......................loss=linear, n_estimators=70; total time=   0.8s
[CV] END .......................loss=linear, n_estimators=60; total time=   1.0s
[CV] END .......................loss=linear, n_estimators=50; total time=   1.1s
[CV] END .......................loss=linear, n_estimators=60; total time=   1.2s
[CV] END .......................loss=linear, n_estimators=50; total time=   1.2s
[CV] END .......................loss=linear, n_estimators=50; total time=   1.5s
[CV] END .......................loss=linear, n_estimators=60; total time=   2.1s
[CV] END .......................loss=linear, n_estimators=70; total time=   1.2s
[CV] END .......................loss=linear, n_estimators=80; total time=   1.1s
[CV] END .......................loss=linear, n_estimators=80; total time=   1.2s
[CV] END .......................loss=linear, n_estimators=70; total time=   2.6s
[CV] END .......................loss=square, n_e

In [20]:
model = AdaBoostRegressor(n_estimators = 70, loss = "linear")
model.fit(X_train, y_train) 
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

In [21]:
print('Model performance for Training set')
print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
print("- R2 Score: {:.4f}".format(model_train_r2))

print('----------------------------------')

print('Model performance for Test set')
print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
print("- R2 Score: {:.4f}".format(model_test_r2))
print('----------------------------------')

Model performance for Training set
- Root Mean Squared Error: 502927.4537
- Mean Absolute Error: 409426.8838
- R2 Score: 0.6881
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 533106.9469
- Mean Absolute Error: 427256.4619
- R2 Score: 0.6225
----------------------------------
